In [14]:
import cx_Oracle
import pandas as pd
import os
import re

# =========================
# 0) 경로/파일
# =========================
base_path = r"C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종"
excel_file = os.path.join(base_path, "펀드 비중_사학연금.xlsx")  # 원본 비중 파일

# =========================
# 1) 기간 suffix 생성 함수 (기준일 기반)
# - 단일 월: (2025.12)
# - 동일 연도 다중월: (2025.07~12)
# - 연도 넘어감: (2025.11~2026.02)
# =========================
def make_suffix_from_dates(df: pd.DataFrame, date_col: str = "기준일") -> str:
    if date_col not in df.columns:
        return "(result)"

    s = df[date_col].dropna()

    if s.empty:
        return "(result)"

    # 숫자/문자 혼재 대비 → 8자리 YYYYMMDD로 정규화
    s = s.astype(str).str.replace(r"\.0$", "", regex=True).str.strip()
    s = s[s.str.match(r"^\d{8}$")]  # 8자리 아닌 값은 제외

    if s.empty:
        return "(result)"

    start = s.min()  # YYYYMMDD 문자열이므로 min/max 가능
    end = s.max()

    sy, sm = start[:4], start[4:6]
    ey, em = end[:4], end[4:6]

    if sy == ey:
        if sm == em:
            return f"({sy}.{sm})"
        return f"({sy}.{sm}~{em})"
    return f"({sy}.{sm}~{ey}.{em})"

# =========================
# 2) 엑셀 로드 → suffix 결정 (추적의 기준점)
# =========================
df_excel = pd.read_excel(excel_file, sheet_name="Sheet1")
suffix = make_suffix_from_dates(df_excel, date_col="기준일")
print("📌 파일명 suffix:", suffix)

# =========================
# 3) 오라클 조회
# =========================
dsn = cx_Oracle.makedsn("192.168.1.130", 1521, "FIMS2005")
db = cx_Oracle.connect("12512", "lma#12512", dsn)  # 필요시 수정
cursor = db.cursor()

sql = """
SELECT FUND_CD, ZEROIN_TYPE_CD, BM_CD1
FROM TMB002MC
WHERE ZEROIN_TYPE_GB = 'C2'
  AND MEMB_CD= '1061'
  AND END_YMD = '99999999'
  AND ZEROIN_TYPE_CD IN ('A01A','A01G','B01A','B01F1','B01F2','B01G','B01F3', 'B01F4', 'B01F5')
  AND CLOSE_YMD IS NULL
ORDER BY ZEROIN_TYPE_CD
"""
cursor.execute(sql)

columns = [col[0] for col in cursor.description]
df_sql = pd.DataFrame(cursor.fetchall(), columns=columns)

cursor.close()
db.close()

# df_sql.to_excel(sql_out, index=False)
# print("✅ SQL 저장:", sql_out)

# =========================
# 4) 병합 및 컬럼 업데이트
# =========================
df_merged = df_excel.merge(
    df_sql[['FUND_CD', 'BM_CD1', 'ZEROIN_TYPE_CD']],
    left_on='펀드코드',
    right_on='FUND_CD',
    how='inner' #LEFT -> INNER 변경하여 DB에 있는(운용중인) 펀드만 남기기
)

df_merged['BM코드'] = df_merged['BM_CD1']
df_merged['데이터구분'] = df_merged['ZEROIN_TYPE_CD']

df_merged = df_merged.drop(columns=['FUND_CD', 'BM_CD1', 'ZEROIN_TYPE_CD'])

# =========================
# 5) 최종 저장(기간 suffix 포함)
# =========================
out_path = os.path.join(base_path, f"펀드 비중_사학연금_{suffix}.xlsx")
df_merged.to_excel(out_path, index=False)
print("✅ 최종 저장:", out_path)


📌 파일명 suffix: (2025.10~12)
✅ 최종 저장: C:\Users\user\Desktop\업무\2. 파이썬_업무\5. 사학연금 펀드 및 BM 비중 자동화_최종\펀드 비중_사학연금_(2025.10~12).xlsx
